In the next cell, I will include all packages installed for the whole project

In [15]:
#install.packages("Boruta")
#install.packages("cluster")
#install.packages("randomForest")

In [2]:
version$version.string

[1] "R version 4.3.0 (2023-04-21 ucrt)"

We get the data and take a quick look at it

In [5]:
data <- read.csv("datasets/ProjectFiles/train_values.csv")
labels <- read.csv("datasets/ProjectFiles/train_labels.csv")

In [6]:
head(data)
head(labels)
dim(data)
dim(data) - dim(na.omit(data))
#There is no missing data in the whole dataset

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,⋯,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,802906,6,487,12198,2,30,6,5,t,r,⋯,0,0,0,0,0,0,0,0,0,0
2,28830,8,900,2812,2,10,8,7,o,r,⋯,0,0,0,0,0,0,0,0,0,0
3,94947,21,363,8973,2,10,5,5,t,r,⋯,0,0,0,0,0,0,0,0,0,0
4,590882,22,418,10694,2,10,6,5,t,r,⋯,0,0,0,0,0,0,0,0,0,0
5,201944,11,131,1488,3,30,8,9,t,r,⋯,0,0,0,0,0,0,0,0,0,0
6,333020,8,558,6089,2,10,9,5,t,r,⋯,1,0,0,0,0,0,0,0,0,0


,building_id,damage_grade
,<int>,<int>
1,802906,3
2,28830,2
3,94947,3
4,590882,2
5,201944,3
6,333020,2


[1] 260601     39

[1] 0 0

We will be using some different features selection techniques, since there are lots of them. Since the dataset is very big, some of this methods are computationally very expensive. Therefore, I choose to run this methods on a significantly big sample of the whole dataset

In [18]:
features <- subset(data, select = -c(building_id))
response <- labels[, -1]
set.seed(123)
n <- nrow(data)
rows <- sample(n, 500)
sample_data <- data[rows, -1]
sample_response <- labels[rows,-1]


Here, we will mix different feature selection methods. These will be Boruta, Random Forests and Clustering

First, we use the correlation with the output

In [19]:
#Compute correlation across all the input variables and the target variable
correlation_vector <-abs(cor(features,response))
#Rank variables according to correlation with the output
correlation_ranking_idx <-sort(correlation_vector,dec=T,index.return=T)$ix

ERROR: Error in cor(features, response): 'x' must be numeric


In [9]:
library(cluster)
library(Boruta)
#library(randomForest)
#We calculate a distance matrix with daisy, using the method gower, that allows for different data types as is in our case
#dist <- daisy(sample_data, metric = "gower")

#We calculate the best number of clusters maximizing the silouhette width
#set.seed(123)
#num_clusters <- 5:10
#sil_width <- c()
#for (i in num_clusters) {
#  pam_fit <- pam(dist, k = i)
#  sil_width[i] <- pam_fit$silinfo$avg.width
#}
#num_clusters <- which.max(na.omit(sil_width))

#clusters <- pam(dist, k = num_clusters, cluster.only = TRUE)

#Determining most important features using Boruta
boruta_model <- Boruta(sample_response ~ ., data=sample_data, doTrace=0)
boruta_selected_features <- getSelectedAttributes(boruta_model, withTentative=FALSE)
boruta_selected_features

#Determining the most important features using the cluster importance measure
#cluster_importance <- importance(clusters, sample_data)
#cluster_selected_features <- row.names(cluster_importance)[cluster_importance$Overall > 0]
#cluster_selected_features



[1] "geo_level_1_id"                        
 [2] "age"                                   
 [3] "area_percentage"                       
 [4] "height_percentage"                     
 [5] "foundation_type"                       
 [6] "roof_type"                             
 [7] "ground_floor_type"                     
 [8] "has_superstructure_mud_mortar_stone"   
 [9] "has_superstructure_cement_mortar_brick"
[10] "has_superstructure_bamboo"             
[11] "has_superstructure_rc_non_engineered"  
[12] "has_superstructure_rc_engineered"

In [7]:
#filtered_data <- data[,boruta_selected_features]
#head(filtered_data)

ERROR: Error in eval(expr, envir, enclos): objeto 'boruta_selected_features' no encontrado


Even though I couldn't complete feature selection as I wanted, I'll go on
I will try to build one model for prediction